In [28]:
import socket
import sys
import struct
import array
import binascii

# Create a UDP socket



def getMilightSession(server_address):
    sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
    
    message = array.array('B', [0x20 ,0x00 ,0x00 ,0x00 ,0x16 ,0x02 ,0x62 ,0x3A ,0xD5, 0xED, 0xA3, 
               0x01, 0xAE, 0x08, 0x2D, 0x46, 0x61, 0x41, 0xA7, 0xF6, 0xDC, 0xAF, 0xD3, 0xE6, 0x00, 0x00, 0x1E])



    # Send data
    print >>sys.stderr, 'sending "%s"' % message
    sent = sock.sendto(message, server_address)

    # Receive response
    print >>sys.stderr, 'waiting to receive'
    data, server = sock.recvfrom(5987)
    MAC = binascii.hexlify(data[7:13])
 
    vals=struct.unpack('B' * len(data), data)
    print >>sys.stderr, 'received "%s"' % '{}'.format(', '.join(hex(x) for x in vals))

    MAC='{}'.format (':'.join(hex(x) for x in vals[7:13]))
    # sessionID1 = int(data[20])
    sessionID1=vals[19]
    sessionID2 = vals[20]

    return (sock,sessionID1,sessionID2,MAC)


commands={'on':[0x31,0x00,0x00,0x08,0x04,0x01,0x00,0x00,0x00],
          'off':[0x31,0x00,0x00,0x08,0x04,0x02,0x00,0x00,0x00],
          'nighton':[0x31,0x00,0x00,0x08,0x04,0x05,0x00,0x00,0x00],
          'whighton':[0x31,0x00,0x00,0x08,0x05,0x64,0x00,0x00,0x00],

          }

def getCommandPacketRGBW(sess1,sess2,command,zone,value=100,seq=0):
    # lets produce a packet based on these
    header=[0x80,0,0,0,0x11,sess1,sess2,0x00,seq,0]
    output = header+commands[command]
    output.append(zone)
    checksum = sum(output[:-9]) & 0xff
    print 'Checksum = 0x%x' % checksum
    output.append(0)
    output.append(checksum)
    return  (output)

server_address = ('192.168.0.34', 5987)

mysock,WB1,WB2,MAC=getMilightSession(server_address)

print "MAC=", MAC
print "SessionID 0x%x 0x%x" % (WB1,WB2)

message=getCommandPacketRGBW(WB1,WB2,'on',0,0)

print "sending message on"
print '[{}]'.format(', '.join(hex(x) for x in message))

sent = mysock.sendto(array.array('B',message), server_address)

response, server = mysock.recvfrom(5987)
vals=struct.unpack('B' * len(response), response)

formatted='{}'.format(', '.join(hex(x) for x in vals))
print >>sys.stderr, 'received "%s"' % formatted

mysock.close()

MAC= 0xf0:0xfe:0x6b:0x14:0xba:0x9e
SessionID 0x109 0x0
Checksum = 0x2f
sending message on
[0x80, 0x0, 0x0, 0x0, 0x11, 0x6d, 0x0, 0x0, 0x0, 0x0, 0x31, 0x0, 0x0, 0x8, 0x4, 0x1, 0x0, 0x0, 0x0, 0x0, 0x0, 0x2f]


sending "array('B', [32, 0, 0, 0, 22, 2, 98, 58, 213, 237, 163, 1, 174, 8, 45, 70, 97, 65, 167, 246, 220, 175, 211, 230, 0, 0, 30])"
waiting to receive
received "0x28, 0x0, 0x0, 0x0, 0x11, 0x0, 0x2, 0xf0, 0xfe, 0x6b, 0x14, 0xba, 0x9e, 0x42, 0x14, 0x22, 0xf0, 0x0, 0x1, 0x6d, 0x0, 0x0"
received "0x88, 0x0, 0x0, 0x0, 0x3, 0x0, 0x0, 0x0"
